In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_csv('../../dataset/meal_service/train.csv')
df_test = pd.read_csv('../../dataset/meal_service/test.csv')
df_all = pd.concat([df_train, df_test])

In [3]:
def seperate_and_processing_menu_str(menu_row):
    # split by space
    splits = menu_row.split(' ')
    menu = []
    is_new_menu = False
    for dish in splits:
        if len(dish) > 1:
            # find (New)
            if '(New)' in dish:
                menu.append(dish.split('(New)')[0] + dish.split('(New)')[1])
                is_new_menu = True
            elif '(' not in dish and ')' not in dish:
                menu.append(dish)
    if len(menu) <= 3:
        # no menu today
        menu = ['-']
    
    return menu, is_new_menu

In [4]:
def get_menu_info(df):
    is_new_lunch_menu = []
    is_new_dinner_menu = []
    lunch_menus = []
    dinner_menus = []
    menus = []
    for i in range(0, len(df)):
        row_lunch = df.iloc[i]['중식메뉴']
        row_dinner = df.iloc[i]['석식메뉴']
        
        lunch_menu, new_lunch_menu = seperate_and_processing_menu_str(row_lunch)
        is_new_lunch_menu.append(1) if new_lunch_menu else is_new_lunch_menu.append(0)
        
        dinner_menu, new_dinner_menu = seperate_and_processing_menu_str(row_dinner)
        is_new_dinner_menu.append(1) if new_dinner_menu else is_new_dinner_menu.append(0)
        
        if len(lunch_menu) > 1:
            menus.append(lunch_menu)
        if len(dinner_menu) > 1:
            menus.append(dinner_menu)
        lunch_menus.append(lunch_menu)
        dinner_menus.append(dinner_menu)
    
    return is_new_lunch_menu, is_new_dinner_menu, menus, lunch_menus, dinner_menus

In [5]:
is_new_lunch_menu, is_new_dinner_menu, menus, lunch_menus, dinner_menus = get_menu_info(df_train)

In [6]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

try:
    model = Word2Vec.load('food_embedding2.model')
    print('Model loaded')
except:
    print('Training word2vector')
    model = Word2Vec(sentences=menus, vector_size=200, window=7, min_count=0, workers=4, sg=0, epochs=5000)
    model.save('food_embedding2.model')

Training word2vector


In [7]:

a =model.wv.get_vector('된장찌개')
a

array([ 2.5621998 , -2.9234662 , -1.8462129 ,  0.747244  ,  1.2531886 ,
       -2.826936  , -0.4192492 , -1.7955477 , -0.4805617 , -0.4558589 ,
        2.4809537 , -2.5492058 ,  0.22512482, -0.39082628,  1.5270308 ,
       -0.3601735 ,  1.1617361 ,  0.6410036 , -0.78434134, -1.587272  ,
       -1.391548  , -0.78139514,  4.4507127 ,  0.82720363,  0.23091164,
        1.2979548 , -0.9594517 ,  4.211911  ,  0.31241018, -0.3357658 ,
        1.8037072 , -1.0004157 , -1.3151643 , -0.5957134 , -1.0063505 ,
       -1.9670038 , -1.5172161 ,  1.9287276 ,  0.7961582 , -0.3059118 ,
       -1.1755251 , -1.7106228 ,  0.29821345, -0.3805697 , -1.1606536 ,
        1.3811921 , -0.71667075, -1.2156348 ,  2.0016675 , -0.08737516,
       -1.0720786 , -1.8707728 , -3.1563582 ,  0.25879544,  0.5672593 ,
        0.7546911 , -2.3873246 , -0.44362482, -0.6848357 ,  1.2052848 ,
        2.6414297 , -0.28164858,  1.8826356 ,  0.5459539 , -0.81961304,
        2.425498  ,  0.12786618, -1.3171816 , -1.2053521 ,  2.34

In [8]:
type(a)

numpy.ndarray

In [9]:
b = np.array([0] * 200)
b


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [10]:
def menu_to_vectors(model, menus):
    soups = []
    mains = []
    for menu in menus:
        if len(menu) > 2:
            soup = menu[1]
            main = menu[2]
            soups.append(model.wv.get_vector(soup))
            mains.append(model.wv.get_vector(main))
        else:
            soups.append(np.array([0] * 200))
            main.append(np.array([0] * 200))
    
    return soups, mains

In [11]:
lunch_soups_list, lunch_mains_list = menu_to_vectors(model, lunch_menus)

In [12]:
lunch_soups = pd.DataFrame({'lunch_soups': lunch_soups_list})

In [13]:
lunch_soups

,lunch_soups
0,"[0.7157718, 0.17222802, -1.4951952, 0.29216596..."
1,"[-0.8692188, -1.4180783, -0.5114756, -0.219945..."
2,"[2.0050225, -0.90456784, -0.56100386, -1.67317..."
3,"[-0.96780944, -0.43067113, -2.8945706, 0.43301..."
4,"[0.34467676, -2.0340931, -4.163531, 0.06545718..."
...,...
1200,"[-0.18243644, -0.12082305, -2.0283463, -0.6465..."
1201,"[0.5375248, -1.6184533, -0.64022994, 0.6134748..."
1202,"[0.4657313, -1.2884558, -1.6994308, 0.6908728,..."
1203,"[0.70798653, -0.19663419, -0.8548546, -0.12540..."
